In [23]:
import re
import pandas as pd

In [24]:
def isolated_positive(text):
    pt = r'[(](\d{1,2})/(\d{1,2})[)]'
    #pt = r'(\d{1,2})/(\d{1,2})'
    y = re.findall(pt, text)
    if y == []:
        return ''
    else:
        return y[0]
    
def pos_iso(result):
    try:
        (positive, isolated) = result
        return int(positive), int(isolated)
    except:
        return '', ''

In [25]:
# remove digit
def dot1(text):
    ol = re.compile('(\d+)[.]')
    ol = ol.sub(' ', text)
    return ol
def para1(text):
    ol = re.compile('(\d+)[)]')
    ol = ol.sub(' ', text)
    return ol
def para2(text):
    ol = text.split(' ', 1)
    if ol[0].isdigit():
        return ol[1]
def remove_digit(text):
    text = dot1(text)
    text = para1(text)
    #text = para2(text)
    text = text.replace('.', '')
    return " ".join(text.split())

# remove roman
u1 = ['I', 'II','III', 'IV', 'V', 'VI', 'VII', 'VIII','IX', 'X']
u2 = ['a', 'b','c', 'd', 'e', 'f', 'g', 'h','i', 'j']
unit = u1 + u2 
#regex = r'(?:' + '|'.join(unit) + r')'
regex = r'(' + '|'.join(unit) + r')'
print(regex)

def dot(text):
    n=5
    text1 = text[:n]
    u_text = re.sub(regex + r'[.]' ,'', text1)+text[n:]
    return u_text
def para(text):
    n=5
    text1 = text[:n]
    text1 = text1.replace('(', '')
    u_text = re.sub(regex + r'[)]' ,'', text1)+text[n:]
    return u_text

def other(text):
    ol = text.split(' ', 1)
    if ol[0] in unit:
        return ol[1]
    else:
        return text  
    
def remove_roman(text):
    text = dot(text)
    text = para(text)
    text = other(text)
    text = text.replace('.', '')
    return text.strip()

# extract preceding word
def prev_word1(text):
    text = remove_roman(text)
    text = remove_digit(text)
    #print("text: ", text)
    if text.find(':')!=-1:
        s3 = text.split(':')
        if s3[0] == 'LYMPHNODES' and len(s3)>2:
            s3 = s3[1]
            s3 = re.sub(' +', ' ', s3).strip()
            return s3
        else:
            return s3[0]

    elif text.find('-')!=-1:
        s3 = text.split('-')[0]
        s3 = re.sub(' +', ' ', s3).strip()
        return s3
    
    elif text.find(';')!=-1:
        s3 = text.split(';')[0]
        s3 = re.sub(' +', ' ', s3).strip()
        return s3


(I|II|III|IV|V|VI|VII|VIII|IX|X|a|b|c|d|e|f|g|h|i|j)


In [26]:
def main(text): 
    prev_word = []
    positive = []
    isolated = []
    txt = []
    # apply on loop    
    for i in range(len(text)):
        t = text[i]
        # t = replace(t)
        #print(t)
        result = isolated_positive(t)
        #print(result)
        #print((result))
        pos, iso = pos_iso(result)
        #print(pos, 't: ', iso)
        if len(result)>0:
            # prev word
            word3 = prev_word1(t)
            if word3 is not None:
                word3 = word3.replace('-', '').strip()
            #print(word3)
            if (word3==None) and ":" in text[i-1]:
                t = text[i-1]
                t = t + text[i]
                #print("printed new text = ",t)
                word3 = prev_word1(t) 
                if word3 is not None:
                    word3 = word3.replace('-', '').strip()
            # append numbers
            positive.append(pos)
            isolated.append(iso)
            prev_word.append(word3)
            txt.append(t)
            # print(t)
    df = pd.DataFrame(positive, columns =['positive'])
    df['isolated'] = isolated
    df['lymph_prev_word'] = prev_word
    #df['file'] = file
    df['text'] = txt 
    return df

In [27]:
# version 2 : run script on database
def solution(data):
    # extract name and text row wise and calculate results
    def name_text(value):
        text = data['RESULT_VALUE_C'][value]
        text = text.split('\n')
        name = data['name'][value]
        return text, name
    # create new dataframe and push all the results for 200 examples in new dataset
    new_df=pd.DataFrame(columns=['positive','isolated','lymph_prev_word','name', 'text'])
    for i in range(data.shape[0]):
        text, name = name_text(i)
        df = main(text)
        df['name'] = name
        new_df = new_df.append(df)
    # re arrange data columns
    new_df = new_df[['name', 'positive','isolated','lymph_prev_word', 'text']]
    return new_df

In [30]:
# input data
path = "Lymphnode200.csv"
data = pd.read_csv(path , escapechar='\\')

In [31]:
# output data
new = solution(data)

C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureW

C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureW

C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureW

C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureW

C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(df)
C:\Users\shweta\AppData\Local\Temp\ipykernel_16588\1173096661.py:15: FutureW

In [32]:
print(new.shape)
new.head(3)

(289, 5)


,name,positive,isolated,lymph_prev_word,text
0,MANGO_LYMPHNODE_REPORT_1,0,6,5 RIGHT PELVIC LYMPHNODES,"5. RIGHT PELVIC LYMPHNODES:6 NODES ISOLATED,AL..."
1,MANGO_LYMPHNODE_REPORT_1,0,7,6 LEFT PELVIC LYMPHNODES,"6. LEFT PELVIC LYMPHNODES: 7 NODES ISOLATED, A..."
2,MANGO_LYMPHNODE_REPORT_1,0,3,8 RETROPERITONEAL LYMPHNODES,8. RETROPERITONEAL LYMPHNODES: 3 LYMPHNODES IS...


In [29]:
# export output
#new.to_csv("solutionv6.csv", index=False)